In [1]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas

In [2]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

In [3]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [4]:
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": [37.04346, 33.52843, 42.91386, 38.627, 41.47342],
	"longitude": [-76.36711, -112.08882, -78.82405, -90.1994, -81.66491],
	"start_date": "2008-01-01",
	"end_date": "2023-12-31",
	"daily": ["temperature_2m_max", "temperature_2m_mean", "temperature_2m_min", "apparent_temperature_max", "apparent_temperature_mean", "apparent_temperature_min", "rain_sum", "snowfall_sum"],
	"temperature_unit": "fahrenheit",
	"precipitation_unit": "inch",
	"timezone": "auto"
}

In [5]:
responses = openmeteo.weather_api(url, params=params)

In [6]:
# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
daily_temperature_2m_mean = daily.Variables(2).ValuesAsNumpy()
daily_apparent_temperature_max = daily.Variables(3).ValuesAsNumpy()
daily_apparent_temperature_min = daily.Variables(4).ValuesAsNumpy()
daily_apparent_temperature_mean = daily.Variables(5).ValuesAsNumpy()
daily_rain_sum = daily.Variables(6).ValuesAsNumpy()
daily_snowfall_sum = daily.Variables(7).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["apparent_temperature_max"] = daily_apparent_temperature_max
daily_data["apparent_temperature_min"] = daily_apparent_temperature_min
daily_data["apparent_temperature_mean"] = daily_apparent_temperature_mean
daily_data["rain_sum"] = daily_rain_sum
daily_data["snowfall_sum"] = daily_snowfall_sum


Coordinates 37.01230239868164°N -76.34564208984375°E
Elevation 2.0 m asl
Timezone b'America/New_York' b'EDT'
Timezone difference to GMT+0 -14400 s


In [7]:
daily_dataframe = pd.DataFrame(data = daily_data)
print(daily_dataframe)

                          date  temperature_2m_max  temperature_2m_min  \
0    2008-01-01 04:00:00+00:00           56.401703           47.285450   
1    2008-01-02 04:00:00+00:00           38.581699           34.010452   
2    2008-01-03 04:00:00+00:00           33.091702           29.045454   
3    2008-01-04 04:00:00+00:00           40.381699           29.937952   
4    2008-01-05 04:00:00+00:00           46.231701           35.765450   
...                        ...                 ...                 ...   
5839 2023-12-27 04:00:00+00:00           62.251701           57.170452   
5840 2023-12-28 04:00:00+00:00           59.911701           55.865448   
5841 2023-12-29 04:00:00+00:00           54.781700           48.234192   
5842 2023-12-30 04:00:00+00:00           49.291702           43.906696   
5843 2023-12-31 04:00:00+00:00           48.031700           41.570454   

      temperature_2m_mean  apparent_temperature_max  apparent_temperature_min  \
0               39.841698     

In [8]:
daily_dataframe

,date,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,rain_sum,snowfall_sum
0,2008-01-01 04:00:00+00:00,56.401703,47.285450,39.841698,48.391617,38.937428,28.266708,0.000000,0.000000
1,2008-01-02 04:00:00+00:00,38.581699,34.010452,30.571699,26.994125,20.169397,14.985704,0.003937,0.192913
2,2008-01-03 04:00:00+00:00,33.091702,29.045454,25.981701,19.120697,14.817845,9.800150,0.000000,0.000000
3,2008-01-04 04:00:00+00:00,40.381699,29.937952,22.651699,30.081749,20.549417,14.580448,0.000000,0.000000
4,2008-01-05 04:00:00+00:00,46.231701,35.765450,27.871700,35.756351,25.655077,17.346970,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
5839,2023-12-27 04:00:00+00:00,62.251701,57.170452,52.801701,58.690948,53.370998,48.684361,0.346457,0.000000
5840,2023-12-28 04:00:00+00:00,59.911701,55.865448,52.621700,57.768600,53.430882,49.283543,0.003937,0.000000
5841,2023-12-29 04:00:00+00:00,54.781700,48.234192,42.631699,52.702675,44.318356,38.567394,0.173228,0.000000
5842,2023-12-30 04:00:00+00:00,49.291702,43.906696,38.491699,39.206577,35.123623,30.011618,0.094488,0.000000


In [9]:
daily_dataframe['date'] = pd.to_datetime(daily_dataframe['date'])

In [10]:
daily_dataframe['year'] = daily_dataframe['date'].dt.year
daily_dataframe['date'] = daily_dataframe['date'].dt.strftime('%m-%d')

In [11]:
# Calculate the number of rows in the DataFrame
num_rows = daily_dataframe.shape[0]

# Create lists of latitude and longitude values with the correct length
latitude_values = [37.04346, 33.52843, 42.91386, 38.627, 41.47342] * (num_rows // 5) + [37.04346] * (num_rows % 5)
longitude_values = [-76.36711, -112.08882, -78.82405, -90.1994, -81.66491] * (num_rows // 5) + [-76.36711] * (num_rows % 5)

# Assign the values to new columns in the DataFrame
daily_dataframe['latitude'] = latitude_values
daily_dataframe['longitude'] = longitude_values

In [12]:
daily_dataframe

,date,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,rain_sum,snowfall_sum,year,latitude,longitude
0,01-01,56.401703,47.285450,39.841698,48.391617,38.937428,28.266708,0.000000,0.000000,2008,37.04346,-76.36711
1,01-02,38.581699,34.010452,30.571699,26.994125,20.169397,14.985704,0.003937,0.192913,2008,33.52843,-112.08882
2,01-03,33.091702,29.045454,25.981701,19.120697,14.817845,9.800150,0.000000,0.000000,2008,42.91386,-78.82405
3,01-04,40.381699,29.937952,22.651699,30.081749,20.549417,14.580448,0.000000,0.000000,2008,38.62700,-90.19940
4,01-05,46.231701,35.765450,27.871700,35.756351,25.655077,17.346970,0.000000,0.000000,2008,41.47342,-81.66491
...,...,...,...,...,...,...,...,...,...,...,...,...
5839,12-27,62.251701,57.170452,52.801701,58.690948,53.370998,48.684361,0.346457,0.000000,2023,41.47342,-81.66491
5840,12-28,59.911701,55.865448,52.621700,57.768600,53.430882,49.283543,0.003937,0.000000,2023,37.04346,-76.36711
5841,12-29,54.781700,48.234192,42.631699,52.702675,44.318356,38.567394,0.173228,0.000000,2023,37.04346,-76.36711
5842,12-30,49.291702,43.906696,38.491699,39.206577,35.123623,30.011618,0.094488,0.000000,2023,37.04346,-76.36711


In [13]:
# lookup dictionary for latitude and longitude to city mapping
lat_lon_to_city = {
    (37.04346, -76.36711): 'Hampton',
    (33.52843, -112.08882): 'Phoenix',
    (42.91386, -78.82405): 'Buffalo',
    (38.627, -90.1994): 'Saint Louis',
    (41.47342, -81.66491): 'Cleveland'
}

In [14]:
# function to lookup city based on latitude and longitude
def get_city(lat, lon):
    return lat_lon_to_city.get((lat, lon), 'Unknown')

In [15]:
# create a new city column
daily_dataframe['city'] = daily_dataframe.apply(lambda row: get_city(row['latitude'], row['longitude']), axis=1)

In [16]:
# Remove the latitude, longitude, and date columns
daily_dataframe = daily_dataframe.drop(columns=['latitude', 'longitude', 'date'])

In [17]:
# Rearrange the columns to show city, date, year, temps from max to min, rain, and snowfall sum
columns_order = ['city', 'year', 'temperature_2m_max', 'temperature_2m_mean', 'temperature_2m_min', 'apparent_temperature_max', 'apparent_temperature_mean', 'apparent_temperature_min', 'rain_sum', 'snowfall_sum']

In [18]:
# defining columns under a new dataframe name to show the diesred layout for output
new_layout = daily_dataframe[columns_order]

In [19]:
# Rename multiple columns
new_layout.rename(columns={
    'rain_sum': 'rain',
    'snowfall_sum': 'snowfall'
}, inplace=True)

In [20]:
new_layout

,city,year,temperature_2m_max,temperature_2m_mean,temperature_2m_min,apparent_temperature_max,apparent_temperature_mean,apparent_temperature_min,rain,snowfall
0,Hampton,2008,56.401703,39.841698,47.285450,48.391617,28.266708,38.937428,0.000000,0.000000
1,Phoenix,2008,38.581699,30.571699,34.010452,26.994125,14.985704,20.169397,0.003937,0.192913
2,Buffalo,2008,33.091702,25.981701,29.045454,19.120697,9.800150,14.817845,0.000000,0.000000
3,Saint Louis,2008,40.381699,22.651699,29.937952,30.081749,14.580448,20.549417,0.000000,0.000000
4,Cleveland,2008,46.231701,27.871700,35.765450,35.756351,17.346970,25.655077,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
5839,Cleveland,2023,62.251701,52.801701,57.170452,58.690948,48.684361,53.370998,0.346457,0.000000
5840,Hampton,2023,59.911701,52.621700,55.865448,57.768600,49.283543,53.430882,0.003937,0.000000
5841,Hampton,2023,54.781700,42.631699,48.234192,52.702675,38.567394,44.318356,0.173228,0.000000
5842,Hampton,2023,49.291702,38.491699,43.906696,39.206577,30.011618,35.123623,0.094488,0.000000


In [21]:
# created a new data frame by grouping city and year with rain and snowfall to toatl up the sum
grouped_df = new_layout.groupby(['city', 'year'])[['rain', 'snowfall']].sum().reset_index()

In [22]:
grouped_df

,city,year,rain,snowfall
0,Buffalo,2008,4.153543,0.110236
1,Buffalo,2009,9.039370,0.000000
2,Buffalo,2010,7.255906,2.287401
3,Buffalo,2011,5.787401,0.055118
4,Buffalo,2012,12.090550,0.110236
...,...,...,...,...
75,Saint Louis,2019,5.059055,1.791339
76,Saint Louis,2020,10.929133,1.322835
77,Saint Louis,2021,7.240157,0.330709
78,Saint Louis,2022,8.523622,1.185039


In [24]:
# will need to re-add Ridership.csv to sample_data folder
df2 = pd.read_csv('/content/sample_data/Ridership.csv', header=0)

In [25]:
df2

,Agency,NTD ID,UPT,Year,VOMS,City,State
0,Niagara Frontier Transportation Authority,20004,6868958,2008,23,Buffalo,NY
1,Niagara Frontier Transportation Authority,20004,6259921,2009,23,Buffalo,NY
2,Niagara Frontier Transportation Authority,20004,5990778,2010,23,Buffalo,NY
3,Niagara Frontier Transportation Authority,20004,6973287,2011,23,Buffalo,NY
4,Niagara Frontier Transportation Authority,20004,6460209,2012,23,Buffalo,NY
...,...,...,...,...,...,...,...
76,"Valley Metro Rail, Inc.",90209,8557187,2020,40,Phoenix,AZ
77,"Valley Metro Rail, Inc.",90209,7250503,2021,38,Phoenix,AZ
78,"Valley Metro Rail, Inc.",90209,8786881,2022,32,Phoenix,AZ
79,"Valley Metro Rail, Inc.",90209,10055291,2023,32,Phoenix,AZ


In [28]:
# mergred dataset Ridership.csv with data frame grouped_df to create a new data frame
merged_df = pd.merge(grouped_df, df2, left_on=['city', 'year'], right_on=['City', 'Year'])

In [29]:
merged_df

,city,year,rain,snowfall,Agency,NTD ID,UPT,Year,VOMS,City,State
0,Buffalo,2008,4.153543,0.110236,Niagara Frontier Transportation Authority,20004,6868958,2008,23,Buffalo,NY
1,Buffalo,2009,9.039370,0.000000,Niagara Frontier Transportation Authority,20004,6259921,2009,23,Buffalo,NY
2,Buffalo,2010,7.255906,2.287401,Niagara Frontier Transportation Authority,20004,5990778,2010,23,Buffalo,NY
3,Buffalo,2011,5.787401,0.055118,Niagara Frontier Transportation Authority,20004,6973287,2011,23,Buffalo,NY
4,Buffalo,2012,12.090550,0.110236,Niagara Frontier Transportation Authority,20004,6460209,2012,23,Buffalo,NY
...,...,...,...,...,...,...,...,...,...,...,...
71,Saint Louis,2019,5.059055,1.791339,Bi-State Development Agency of the Missouri-Il...,70006,13087672,2019,50,Saint Louis,MO
72,Saint Louis,2020,10.929133,1.322835,Bi-State Development Agency of the Missouri-Il...,70006,6746245,2020,50,Saint Louis,MO
73,Saint Louis,2021,7.240157,0.330709,Bi-State Development Agency of the Missouri-Il...,70006,5884125,2021,42,Saint Louis,MO
74,Saint Louis,2022,8.523622,1.185039,Bi-State Development Agency of the Missouri-Il...,70006,6755791,2022,42,Saint Louis,MO


In [30]:
# converted data frame merged_df into a csv
merged_df.to_csv('top_5_cities_weather.csv', index=False)